In [ ]:
import gymnasium
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import CheckpointCallback
from utils.env import CogSatEnv
import os

In [ ]:
channel_size_list = [10,15,20,25,30]
leo_users_list = [10,15,20]
geo_users_list = [10,15,20]


count = 0

for channel_size in channel_size_list:
    for leo_users in leo_users_list:
        for geo_users in geo_users_list:
            saved_folder = f"saved_data_{channel_size}_channels_{leo_users}_leo_user_{geo_users}_geo_user"
            print(saved_folder)  # or use it however you need
            count+=1

print("cpunt",count)


In [ ]:


# set the seed
seed = 42

gymnasium.register(
    id='CogSatEnv-v1',  # Use the same ID here as you used in the script
    entry_point='utils.env:CogSatEnv',
)

# Custom parameters
env_config = {
    "no_channels": 25,
    "no_leo_user": 15,
    "no_geo_user": 10,
    "saved_folder": "saved_data_15_10_10"
}


# Folder name
saved_folder = env_config["saved_folder"]

# Create the folder if it doesn't exist
if not os.path.exists(saved_folder):
    os.makedirs(saved_folder)
    print(f"Folder '{saved_folder}' created.")
else:
    print(f"Folder '{saved_folder}' already exists.")



import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# Initialize the environment
env_id = "CogSatEnv-v1"
env = make_vec_env(env_id, n_envs=1, seed=seed, env_kwargs={"env_config": env_config})

epoch_length = 122 ## got through experiment
epoch_numbers = 4000


# Specify the policy network architecture, here we are using the default MIP
model = A2C("MultiInputPolicy", env, verbose=1,ent_coef=0.01, tensorboard_log="./a2c_dsa_tensorboard/",
            seed=seed)

# Define the total number of timesteps to train the model
total_timesteps = epoch_length*epoch_numbers

# Train the model
model.learn(total_timesteps=total_timesteps)

# Save the model
model.save("a2c_cogsatenv_1")

env.close()